Dependencias

In [ ]:
!pip install mediapipe

In [ ]:
import cv2 as cv
import os
import pickle
import time
import uuid
import mediapipe as mp

Definir clases e imagenes

In [ ]:
IMAGENES_PATH = "imagenes/recopiladas"

In [ ]:
clases = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "l", "m", "n", "o", "p", "r", "s", "t", "u", "v", "w", "y"]
num_imgs = 15

Crear imagenes para base de datos

In [ ]:
for clase in clases:
    os.system(f"mkdir -p {IMAGENES_PATH}/{clase}")
    cap = cv.VideoCapture(0)
    
    if not cap.isOpened():
        print("-- Camara no se pudo abrir --")
        break
    
    print(f"\nRecolectando imágenes para: {clase}")
    
    respuesta = str(input("Ingresa 0 para cerrar, y cualquier tecla para empezar: "))
    if respuesta == "0": 
        print("-- Proceso terminado --")
        cap.release()
        break
        
    print("\n*Recoleccion EMPEZADA*")
    
    for img_num in range(num_imgs):
        print(img_num, end=" ")
        ret, frame = cap.read()
        nom_img = os.path.join(IMAGENES_PATH, clase, clase + "." + f"{str(uuid.uuid1())}.jpg")
        
        cv.imwrite(nom_img, frame)
        cv.imshow("frame", frame)
        time.sleep(0.5)
    
    print("\n\n*Recolección ACABADA*\n")

    cap.release()

Etiquetar imágenes

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [ ]:
sides = ['right', 'left']

data = []
labels = []

for dir_ in os.listdir(IMAGENES_PATH):
    for img_path in os.listdir(os.path.join(IMAGENES_PATH, dir_)):
        
        img = cv.imread(os.path.join(IMAGENES_PATH, dir_, img_path))
        for side in sides:
            
            data_aux = []

            x_ = []
            y_ = []
            
            img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    for i in range(len(hand_landmarks.landmark)):
                        x = hand_landmarks.landmark[i].x
                        y = hand_landmarks.landmark[i].y
    
                        x_.append(x)
                        y_.append(y)
    
                    min_x = min(x_)
                    min_y = min(y_)
                    max_x = max(x_)
                    max_y = max(y_)
                    
                    for i in range(len(hand_landmarks.landmark)):
                        x = hand_landmarks.landmark[i].x
                        y = hand_landmarks.landmark[i].y
                        data_aux.append((x - min_x)/(max_x - min_x))
                        data_aux.append((y - min_y)/(max_y - min_y))
    
                data.append(data_aux)
                labels.append(dir_ + '_' + side)
            
            img = cv.flip(img, 1) # Flip image horizontally to identify the sign with the opposite hand

In [ ]:
f = open('data.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()